# <span><img src="./MavenomicsLogoNew.png" alt="Mavenomics Logo" style="margin-bottom:-15px" />&nbsp;&nbsp;|&nbsp; Rocket Motors</span>

Imagine a hypothetical mission where you had a roll of duct tape, a couple of rocket engines that you bought off the internet, and a complete lack of common sense. Where could you go?

This dashboard answers that by letting you explore the efficiency and power of various rocket engines, and plan a theoretical mission to visit various celestial bodies by duct taping those rockets together. Data for this dashboard is collected from [ThrustCurve](http://thrustcurve.org), a hobbyist database of consumer rocket engines.

Definitions: 

 - $I_{sp}$, or _Specific Impulse_, is a sort of space mileage- it measures how efficient a rocket engine is at turning fuel into thrust. $I_{sp}$ can vary based on pressure, so we assume that you're already in space. Most space engines generally achieve about 300-350s of $I_{sp}$, but the engines we'll be using are at around half that.
 
 - $\Delta V$, or _Delta-V_, measures how much a spacecraft can change it's orbital velocity- a way of judging how far it can go. It takes about 10,000 $\Delta V$ to achieve Earth orbit, and a further 6,000 to get from there to the Moon. As you'll see in this dashboard, that is a staggering amount of energy!
 
Given some information about the rocket's weight, we can calculate $\Delta V$ using the [Tsiolkovsky rocket equation](https://en.wikipedia.org/wiki/Tsiolkovsky_rocket_equation):

 #### $\Delta V = g * I_{sp} * ln \frac{m_0}{m_f}$
 
Where $m_0$ is the rocket's "wet" mass (rocket + fuel), $m_f$ is the rocket's "dry" mass (rocket alone, minus the fuel), and $g$ is the accelleration due to Earth's gravity.

In [ ]:
# Prereqs that aren't already included in the MavenWorks environment:
import sys
!{sys.executable} -m pip install lxml

In [2]:
import requests
import pandas
import math
import numpy as np
import ipywidgets as widgets
import mavenworks
from lxml import html
import xml.etree.ElementTree as ET

In [3]:
# ThrustCurve.org has a public XML API for querying rocket engines
url = "http://www.thrustcurve.org/servlets/search"

def get_motor_data(eng_diam, eng_mfr):
    request = ET.Element("search-request")
    manufacturer = ET.SubElement(request, "manufacturer")
    diameter = ET.SubElement(request, "diameter")

    manufacturer.text = eng_mfr;

    diameter.text = str(eng_diam) # millimeters

    result = requests.post(url, data=ET.tostring(request, encoding="ascii"))
    
    if result.status_code != 200:
        print(result.text)
        return pandas.DataFrame.from_dict([])

    result_data = ET.fromstring(result.text)
    result_table = []

    for motor in result_data[1]:
        motor_entry = {}
        for detail in motor:
            col = detail.tag.split("}")[1]
            motor_entry[col] = detail.text
        result_table.append(motor_entry)

    return pandas.DataFrame.from_dict(result_table)

In [4]:
motors_table = get_motor_data("98", "Cesaroni")
motors_table.head()

,motor-id,manufacturer,manufacturer-abbrev,designation,brand-name,common-name,impulse-class,diameter,length,type,...,data-files,info-url,total-weight-g,prop-weight-g,case-info,prop-info,updated-on,availability,delays,sparky
0,509,Cesaroni Technology,Cesaroni,10133M795-P,Pro98 M795,M795,M,98.0,702.0,reload,...,2,http://www.thrustcurve.org/motorsearch.jsp?id=509,8492.0,4892.0,Pro98-4G,Classic,2019-04-16,regular,NaN,NaN
1,897,Cesaroni Technology,Cesaroni,10347N10000-P,Pro98-6G Vmax,N10000,N,98.0,1010.0,reload,...,2,http://www.thrustcurve.org/motorsearch.jsp?id=897,9918.5,5200.0,Pro98-6G,Vmax,2019-04-16,regular,NaN,NaN
2,947,Cesaroni Technology,Cesaroni,10367N1800-P,"CTI, Cesaroni, Pro-X",N1800,N,98.0,702.0,reload,...,2,http://www.thrustcurve.org/motorsearch.jsp?id=947,9180.0,5727.0,Pro98-4G,White,2019-04-16,regular,P,NaN
3,760,Cesaroni Technology,Cesaroni,11077N2600-P,Pro98-6G 11077N2600 Skidmark,N2600,N,98.0,1010.0,reload,...,2,http://www.thrustcurve.org/motorsearch.jsp?id=760,11482.0,6618.0,Pro98-6G,Skidmark,2019-04-16,regular,NaN,true
4,1003,Cesaroni Technology,Cesaroni,12066N2200-P,CTI N2200-PK-P,N2200,N,98.0,1010.0,reload,...,2,http://www.thrustcurve.org/motorsearch.jsp?id=...,11356.0,6122.0,Pro98-5G,Pink,2019-04-16,regular,P,NaN


In [5]:
def GetEngineDetail(info_url: str):
    # ThrustCurve's API doesn't expose Isp as it's not available on every engine
    # Therefore we have to mine the data from the engine's info page
    
    data_table = requests.get(info_url);

    result_data = html.fromstring(data_table.content)
    result_table = []

    headers = result_data.xpath("//table[@class='listing']/tr[@class='listing']/*[1]/text()")
    data = result_data.xpath("//table[@class='listing']/tr[@class='listing']/*[2]/text()")

    return pandas.DataFrame([list(map(lambda d: d.strip(), data))[1:]], columns=headers[0:(len(data)-1)])

In [6]:
# Testing things out...
GetEngineDetail(motors_table.iloc[0]["info-url"]).iloc[0]["Common Name:"]

'M795'

In [7]:
# Consumer rocket engines come in a set of standard sizes, so we'll use this array to hold them
# There are more, but they're either too small to worry about or non-standard
engine_diams = [
    "29",
    "38",
    "54",
    "75",
    "98",
    "132",
    "152",
    "161"
]

# There's plenty of manufacturers, but not many of them test Isp. as it's not often useful (except to us)
engine_mfrs = [
    "Cesaroni Technology",
    "Aerotech",
    "Animal Motor Works", # Makes reloads for CTI casings
]

In [8]:
def format_info_str(selected_motor):
    motor_info = GetEngineDetail(selected_motor).iloc[0]
    if "Isp:" not in motor_info:
        return f"{motor_info['Common Name:']}: No Data"
    wet_mass = float(motor_info["Total Weight:"][:-1])
    dry_mass = wet_mass - float(motor_info["Prop. Weight:"][:-1])
    i_sp = float(motor_info["Isp:"][:-1])
    dV = (i_sp * 9.8) * math.log(wet_mass / dry_mass)
    return f"{motor_info['Common Name:']}: Isp={i_sp}s dV={dV} m/s"

In [9]:
def calculate_dv_with_mass_ratios(selected_motor):
    motor_info = GetEngineDetail(selected_motor).iloc[0]
    if "Isp:" not in motor_info:
        return pandas.DataFrame(columns=["Dry Mass", "Delta V"])
    wet_mass = float(motor_info["Total Weight:"][:-1])
    dry_mass = wet_mass - float(motor_info["Prop. Weight:"][:-1])
    i_sp = float(motor_info["Isp:"][:-1])
    dV_calc = lambda dead_mass: (i_sp * 9.8) * math.log((wet_mass + dead_mass) / (dry_mass + dead_mass))
    samples = np.arange(0, 10000, 50)
    data = np.fromiter((dV_calc(i) for i in samples), samples.dtype, count=len(samples))
    return pandas.DataFrame({"Dry Mass": samples, "Delta V":data})

In [10]:
def format_question(celestial_body, dry_mass):
    return f"How many engines would you need to go from Low Earth Orbit to {celestial_body} Orbit, with a {dry_mass}g payload?"

CELESTIAL_BODIES = {
    "Lunar": 7150,
    "Martian": 5710,
    "Jovian": 23770,  # _low_ jovian orbit
    "Saturnian": 17940
}

def format_answer(celestial_body, extra_dry_mass, selected_motor):
    motor_info = GetEngineDetail(selected_motor).iloc[0]
    if "Isp:" not in motor_info:
        return f"Try it and find out! (there's no Isp data for {motor_info['Common Name:']})"
    wet_mass = float(motor_info["Total Weight:"][:-1])
    dry_mass = wet_mass - float(motor_info["Prop. Weight:"][:-1])
    i_sp = float(motor_info["Isp:"][:-1])
    dV_calc = lambda dead_mass: (i_sp * 9.8) * math.log((wet_mass + dead_mass) / (dry_mass + dead_mass))
    dV = dV_calc(extra_dry_mass)
    required = CELESTIAL_BODIES[celestial_body]
    return f"You'd need {math.ceil(required / dV)} {motor_info['Common Name:']} engines duct-taped together!"

In [11]:
# Auto-generated code, do not edit!
_json = __import__("json")
display(_json.loads("{\"application/vnd.maven.layout+json\": {\"layout\":{\"properties\":{\"flexSize\":1},\"typeName\":0,\"uuid\":\"76c23bcd-c93b-4dcc-b1bb-56cc1bcbcb5d\",\"attachedProperties\":[{\"Fixed Size (px)\":null,\"Stretch\":0.16801385125760737},{\"Fixed Size (px)\":null,\"Stretch\":1.1953772368164197}],\"children\":[{\"properties\":{\"horizontal\":true,\"prunable\":true},\"typeName\":0,\"uuid\":\"a0ab8701-a7d7-4729-b754-a87a822c7563\",\"attachedProperties\":[{\"Fixed Size (px)\":null,\"Stretch\":1},{\"Fixed Size (px)\":null,\"Stretch\":1}],\"children\":[{\"properties\":{\"caption\":\"Engine Manufacturer\"},\"typeName\":1,\"uuid\":\"526b7c2e-6cb4-4daa-92a0-15ed652f119e\",\"guid\":\"c1c44d79-323f-460b-b6e8-ed89a34fd7de\"},{\"properties\":{\"caption\":\"Engine Diameter (mm)\",\"showRegion\":true},\"typeName\":1,\"uuid\":\"210c539a-6426-40eb-b26c-367d65f9a17f\",\"guid\":\"29900262-2904-43e5-8d5c-2aed0e241e4d\"}]},{\"properties\":{\"ForegroundIndex\":0,\"prunable\":false},\"typeName\":2,\"uuid\":\"e6228ed8-2902-402d-ae83-8a179d00f8c9\",\"attachedProperties\":[{},{},{}],\"children\":[{\"properties\":{\"caption\":\"Select Rocket Engine\",\"prunable\":true},\"typeName\":0,\"uuid\":\"8a95a7e2-5c5e-426a-ba57-9d6e7d74fe8f\",\"attachedProperties\":[{\"Fixed Size (px)\":null,\"Stretch\":1.797424208260572},{\"Fixed Size (px)\":null,\"Stretch\":0.823170731707317}],\"children\":[{\"properties\":{\"caption\":\"SlickGrid\",\"showTitle\":false},\"typeName\":1,\"uuid\":\"0dbf7381-8fc4-4648-b65e-7c70ee09aa0b\",\"guid\":\"b7af02c4-5f5f-4e95-b007-f070481663cf\"},{\"properties\":{\"caption\":\"LabelPart\",\"showTitle\":false},\"typeName\":1,\"uuid\":\"79e28a8f-aee5-4088-b626-35d9b74f08b0\",\"guid\":\"78152a6d-5bcb-4e7c-a60d-a24252335b36\"}]},{\"properties\":{\"caption\":\"Efficiency Loss over Dry Mass (g) from 0 to 10 kg\"},\"typeName\":1,\"uuid\":\"b7d24b72-798a-4e99-a034-2b8993025dba\",\"guid\":\"284f576a-a153-4ea7-aa16-dda73b942e30\"},{\"properties\":{\"caption\":\"Plan your mission!\",\"prunable\":false},\"typeName\":0,\"uuid\":\"02e93b4c-7a14-459c-a28e-f042c75dc9f1\",\"attachedProperties\":[{\"Fixed Size (px)\":null,\"Stretch\":0.7395348837209302},{\"Fixed Size (px)\":null,\"Stretch\":1.2604651162790699}],\"children\":[{\"properties\":{\"horizontal\":true,\"prunable\":true},\"typeName\":0,\"uuid\":\"35070f43-b62e-499b-b815-0efcbbc32351\",\"attachedProperties\":[{\"Fixed Size (px)\":null,\"Stretch\":1.4334082765404341},{\"Fixed Size (px)\":null,\"Stretch\":0.3048127590810089}],\"children\":[{\"properties\":{\"caption\":\"Extra Dry Mass, in grams\"},\"typeName\":1,\"uuid\":\"0f9a1dcb-9e4b-479e-82df-d4f723c9a4b8\",\"guid\":\"e2c607db-19d5-4c21-9d47-dc0f1930559b\"},{\"properties\":{\"caption\":\"Body to Orbit\",\"showRegion\":true},\"typeName\":1,\"uuid\":\"b138d2a5-4838-4a3a-8822-7a3388b6a3c2\",\"guid\":\"7a423c20-4014-4ed0-8aa4-228b25603b8e\"}]},{\"properties\":{\"caption\":\"LabelPart\",\"showTitle\":false},\"typeName\":1,\"uuid\":\"eef88c3a-2b24-4de0-9879-3c5ad2cfc6ef\",\"guid\":\"146bd90e-cf16-4613-9e9c-fc133ce857e4\"}]}]}]},\"parts\":{\"146bd90e-cf16-4613-9e9c-fc133ce857e4\":{\"application/vnd.maven.part+json\":{\"name\":\"LabelPart\",\"id\":\"146bd90e-cf16-4613-9e9c-fc133ce857e4\",\"options\":{\"Value\":{\"type\":\"Eval\",\"expr\":\"format_answer(@OrbitalBody, @ExtraDryMass, @SelectedMotor)\",\"globals\":[\"OrbitalBody\",\"ExtraDryMass\",\"SelectedMotor\"]},\"Caption\":{\"type\":\"Eval\",\"expr\":\"format_question(@OrbitalBody, @ExtraDryMass)\",\"globals\":[\"OrbitalBody\",\"ExtraDryMass\"]}}},\"text/plain\":\"VisualEditorPart\"},\"284f576a-a153-4ea7-aa16-dda73b942e30\":{\"application/vnd.maven.part+json\":{\"name\":\"PivotPart\",\"id\":\"284f576a-a153-4ea7-aa16-dda73b942e30\",\"options\":{\"Input Table\":{\"type\":\"Eval\",\"expr\":\"calculate_dv_with_mass_ratios(@SelectedMotor)\",\"globals\":[\"SelectedMotor\"]},\"Config\":{\"typeName\":\"String\",\"value\":\"{\\\"class\\\":\\\"p-Widget\\\",\\\"plugin\\\":\\\"hypergrid\\\",\\\"row-pivots\\\":\\\"[\\\\\\\"Dry Mass\\\\\\\"]\\\",\\\"column-pivots\\\":\\\"[]\\\",\\\"filters\\\":\\\"[]\\\",\\\"sort\\\":\\\"[]\\\",\\\"style\\\":\\\"position: absolute; width: 100%; height: 100%; z-index: 0; top: 0px; left: 0px;\\\",\\\"view\\\":\\\"d3_y_line\\\",\\\"columns\\\":\\\"[\\\\\\\"Delta V\\\\\\\"]\\\",\\\"aggregates\\\":\\\"{\\\\\\\"Delta V\\\\\\\":\\\\\\\"sum\\\\\\\",\\\\\\\"Dry Mass\\\\\\\":\\\\\\\"sum\\\\\\\"}\\\",\\\"render_time\\\":\\\"26.745000039227307\\\",\\\"updating\\\":\\\"true\\\",\\\"plugin_config\\\":\\\"{}\\\"}\"}}},\"text/plain\":\"VisualEditorPart\"},\"29900262-2904-43e5-8d5c-2aed0e241e4d\":{\"application/vnd.maven.part+json\":{\"name\":\"DropdownPart\",\"id\":\"29900262-2904-43e5-8d5c-2aed0e241e4d\",\"options\":{\"value\":{\"type\":\"Global\",\"expr\":\"EngineDiameter\",\"globals\":[\"EngineDiameter\"]},\"options\":{\"type\":\"Eval\",\"expr\":\"engine_diams\",\"globals\":[]}}},\"text/plain\":\"VisualEditorPart\"},\"78152a6d-5bcb-4e7c-a60d-a24252335b36\":{\"application/vnd.maven.part+json\":{\"name\":\"LabelPart\",\"id\":\"78152a6d-5bcb-4e7c-a60d-a24252335b36\",\"options\":{\"Value\":{\"type\":\"Eval\",\"expr\":\"format_info_str(@SelectedMotor)\",\"globals\":[\"SelectedMotor\"]},\"Caption\":{\"typeName\":\"String\",\"value\":\"Engine Detail\"}}},\"text/plain\":\"VisualEditorPart\"},\"7a423c20-4014-4ed0-8aa4-228b25603b8e\":{\"application/vnd.maven.part+json\":{\"name\":\"DropdownPart\",\"id\":\"7a423c20-4014-4ed0-8aa4-228b25603b8e\",\"options\":{\"value\":{\"type\":\"Global\",\"expr\":\"OrbitalBody\",\"globals\":[\"OrbitalBody\"]},\"options\":{\"type\":\"Eval\",\"expr\":\"list(CELESTIAL_BODIES.keys())\",\"globals\":[]}}},\"text/plain\":\"VisualEditorPart\"},\"b7af02c4-5f5f-4e95-b007-f070481663cf\":{\"application/vnd.maven.part+json\":{\"name\":\"SlickGrid\",\"id\":\"b7af02c4-5f5f-4e95-b007-f070481663cf\",\"options\":{\"Input Table\":{\"type\":\"Mql\",\"expr\":\"/* @EngineDiameter,@EngineMfr */\\nset @cacheKey = 'eng' + @EngineDiameter + @EngineMfr\\nset @table = StaticCache(@cacheKey, KernelEval('get_motor_data(\\\"' + @EngineDiameter + '\\\", \\\"' + @EngineMfr + '\\\")'))\\n\\nSELECT \\n    [common-name] as [Common Name],\\n    [avg-thrust-n] as [Stats.Average Thrust],\\n    [burn-time-s] as [Stats.Burn Time],\\n    [max-thrust-n] as [Stats.Max Thrust],\\n    [prop-weight-g] as [Stats.Propellant Weight],\\n    [tot-impulse-ns] as [Stats.Total Impulse],\\n    [prop-info] as [Propellant Brand],\\n    [impulse-class] as [Impulse Letter Class],\\n    [diameter] as [Engine Diameter],\\n    [info-url] as [ ]\\nFROM ChangeRowName(@table, [designation])\\nGROUP BY \\n    [prop-info],\\n    [impulse-class]\\nWITH ROLLUP\\nHAVING GetLevel() > 0\",\"globals\":[\"EngineDiameter\",\"EngineMfr\"]},\"Formatting\":{\"typeName\":\"String\",\"value\":\"{\\n    \\\" \\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"0\\\"\\n    },\\n    \\\"Common Name\\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"100\\\"\\n    },\\n    \\\"Stats.Average Thrust\\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"90\\\"\\n    },\\n    \\\"Stats.Burn Time\\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"65\\\"\\n    },\\n    \\\"Stats.Max Thrust\\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"70\\\"\\n    },\\n    \\\"Stats.Propellant Weight\\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"105\\\"\\n    },\\n    \\\"Stats.Total Impulse\\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"80\\\"\\n    },\\n    \\\"Propellant Brand\\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"100\\\"\\n    },\\n    \\\"Impulse Letter Class\\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"120\\\"\\n    },\\n    \\\"Engine Diameter\\\": {\\n        \\\"General.ColumnWidthPixels\\\": \\\"100\\\"\\n    }\\n}\\n\"},\"RadioButtonList. Enabled\":{\"typeName\":\"Boolean\",\"value\":true},\"RadioButtonList.Column for Radio Buttons\":{\"typeName\":\"Number\",\"value\":0},\"RadioButtonList.Column for Values\":{\"typeName\":\"Number\",\"value\":10},\"RadioButtonList.Last Selection\":{\"type\":\"Global\",\"expr\":\"SelectedMotor\",\"globals\":[\"SelectedMotor\"]},\"Show Path Column\":{\"typeName\":\"Boolean\",\"value\":true}}},\"text/plain\":\"VisualEditorPart\"},\"c1c44d79-323f-460b-b6e8-ed89a34fd7de\":{\"application/vnd.maven.part+json\":{\"name\":\"DropdownPart\",\"id\":\"c1c44d79-323f-460b-b6e8-ed89a34fd7de\",\"options\":{\"value\":{\"type\":\"Global\",\"expr\":\"EngineMfr\",\"globals\":[\"EngineMfr\"]},\"options\":{\"type\":\"Eval\",\"expr\":\"engine_mfrs\",\"globals\":[]}}},\"text/plain\":\"VisualEditorPart\"},\"e2c607db-19d5-4c21-9d47-dc0f1930559b\":{\"application/vnd.maven.part+json\":{\"name\":\"SliderPart\",\"id\":\"e2c607db-19d5-4c21-9d47-dc0f1930559b\",\"options\":{\"Value\":{\"type\":\"Global\",\"expr\":\"ExtraDryMass\",\"globals\":[\"ExtraDryMass\"]},\"Max\":{\"typeName\":\"Number\",\"value\":10000},\"Step\":{\"typeName\":\"Number\",\"value\":100}}},\"text/plain\":\"VisualEditorPart\"}},\"metadata\":{},\"globals\":[{\"name\":\"EngineDiameter\",\"type\":\"String\",\"value\":\"98\"},{\"name\":\"EngineMfr\",\"type\":\"String\",\"value\":\"Cesaroni Technology\"},{\"name\":\"SelectedMotor\",\"type\":\"String\",\"value\":\"http://www.thrustcurve.org/motorsearch.jsp?id=964\"},{\"name\":\"ExtraDryMass\",\"type\":\"Number\",\"value\":100},{\"name\":\"OrbitalBody\",\"type\":\"String\",\"value\":\"Lunar\"}],\"localParts\":{},\"visual\":true}}"), raw=True)
del _json